In [1]:
from google.colab import drive 
drive.mount('/content/drive')  #brings google drive files to be accessible under /content/

Mounted at /content/drive




---
# Loading data into tfrecords





**Loading a subset of high-res faces**

https://www.kaggle.com/datasets/subinium/highresolution-anime-face-dataset-512x512

In [ ]:
# %%capture
# ! pip install kaggle
# ! mkdir ~/.kaggle
# # ! cp /content/drive/my\ Drive/kaggle.json ~/.kaggle/
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets download subinium/highresolution-anime-face-dataset-512x512

Unzip only 5000 files from zip

https://stackoverflow.com/questions/22243031/unzip-only-limited-number-of-files-in-linux

In [ ]:
# !unzip -Z1 highresolution-anime-face-dataset-512x512.zip | head -5000 | sed 's| |\\ |g' | xargs unzip highresolution-anime-face-dataset-512x512.zip



---


**Download high-res directly from drive**

In [ ]:
%%capture
!unzip -Z1 "/content/drive/My Drive/high_res_anime_faces.zip" | head -10000 | sed 's| |\\ |g' | xargs unzip "/content/drive/My Drive/high_res_anime_faces.zip"

In [2]:
%%capture
!pip install numpy==1.19.5

%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/ZKTKZ/stylegan2.git
# %cd stylegan2
# !nvcc test_nvcc.cu -o test_nvcc -run
!nvcc /content/stylegan2/test_nvcc.cu -o /content/stylegan2/test_nvcc -run

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

!pip install tensorboard

In [ ]:
# convert images to tfrecord
# https://www.tensorflow.org/tutorials/load_data/tfrecord

!python /content/stylegan2/dataset_tool.py create_from_images ./training_data ./portraits 

# !python /content/stylegan2/dataset_tool.py create_from_images \
#     "/content/drive/My Drive/high_res_anime_faces_tfrecords" \
#     ./portraits

Loading images from "./portraits"
detected 10000 images ...
Creating dataset "./training_data"
Adding the images to tfrecords ...
/content/stylegan2/dataset_tool.py:154: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  bytes_list=tf.train.BytesList(value=[quant.tostring()])
Added 10000 images.




---
# Loading pretrained model


https://www.reddit.com/r/MediaSynthesis/comments/gum6f1/tfdne_edit_furry_faces_inbrowser_google_colab/

In [ ]:
# !git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2

In [ ]:
# import gdown
# !wget  -O /content/network-e621.pkl https://thisfursonadoesnotexist.com/model/network-e621-r-512-3194880.pkl

In [15]:
from google_drive_downloader import GoogleDriveDownloader as gdd
model_id = '1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez'  #b/w /d/ and /view/ in 'copy link' of file on drive
# 1WNQELgHnaqMTq3TlrnDaVkyrAH8Zrjez  ##ffhq, realistic human, others drive
# '13zjeOvnZ9pA96sN1yaLWFmK90nRnfYM5'  ##ffhq, realistic human , own drive- pickle error when load
# '1gD6DpzI83E8rTXPwef45FqoE7wF882Eq'  ##metfaces, own drive- pickle error when load

network_pkl = "/content/pretrained.pkl"
gdd.download_file_from_google_drive(file_id=model_id,
                                    dest_path=network_pkl)

In [4]:
# %tensorflow_version 1.4
%cd /content/stylegan2

/content/stylegan2


In [5]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import tensorflow as tf
# import tflex

In [6]:
if 'COLAB_TPU_ADDR' in os.environ:
    os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    os.environ['NOISY'] = '1'

In [10]:
tflib.init_tf()
sess = tf.get_default_session()
sess.list_devices()
# cores = tflex.get_cores()
# tflex.set_override_cores(cores)
# _G0, _D0, Gs0 = pickle.load(open("/content/pretrained.pkl", "rb"))
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
_G0, _D0, Gs0 = pickle.load(open(network_pkl, "rb"))


# _G = Instantaneous snapshot of the generator. Mainly useful for resuming a previous training run.
# _D = Instantaneous snapshot of the discriminator. Mainly useful for resuming a previous training run.
# Gs = Long-term average of the generator. Yields higher-quality results than the instantaneous snapshot.

UnpicklingError: ignored

In [22]:
%cd /content/
import pretrained_networks
network_pkl = "gdrive:networks/stylegan2-ffhq-config-f.pkl"
gdd.download_file_from_google_drive(file_id=model_id,
                                    dest_path=network_pkl)
print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)

/content
Loading networks from "gdrive:networks/stylegan2-ffhq-config-f.pkl"...




---
# Training




https://colab.research.google.com/github/ZKTKZ/thdne/blob/master/StyleGAN2_Tazik_25GB_RAM.ipynb#scrollTo=vmeWpLdZdhq2

In [24]:
# !python /content/stylegan2/run_training.py --num-gpus=1 --data-dir=/content/stylegan2 --config=config-f --dataset=training_data \
# --total-kimg=10000 --result-dir="/content/drive/My Drive/twist_moe/results/" --resume-pkl='/content/network-e621.pkl'

## does not seem to get data from drive:
!python /content/stylegan2/run_training.py --num-gpus=1 --data-dir="/content/drive/My Drive/" \
--config=config-f --dataset=high_res_anime_faces_tfrecords \
--total-kimg=10 --result-dir="/content/drive/My Drive/realfaces_anime/results/" --resume-pkl="/content/gdrive:networks/stylegan2-ffhq-config-f.pkl"


Local submit - run_dir: /content/drive/My Drive/realfaces_anime/results/00003-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 512, 512]
Dynamic range = [0, 255]
Label size    = 0
Loading networks from "/content/gdrive:networks/stylegan2-ffhq-config-f.pkl"...
Traceback (most recent call last):
  File "/content/stylegan2/run_training.py", line 215, in <module>
    main()
  File "/content/stylegan2/run_training.py", line 210, in main
    run(**vars(args))
  File "/content/stylegan2/run_training.py", line 134, in run
    dnnlib.submit_run(**kwargs)
  File "/content/stylegan2/dnnlib/submission/submit.py", line 343, in submit_run
    return farm.submit(submit_config, host_run_dir)
  File "/content/stylegan2/dnnlib/submission/internal/local.py", line 22, in submit
    return run_wrapper(submit_config)
  File "/content/stylegan2/dnnlib/submi

In [ ]:
# !python /content/stylegan2/run_training.py --spatial-augmentations=true --lr=0.0005 --num-gpus=1 \
# --data-dir=./dataset --config=config-f --dataset=hayasaka --mirror-augment=true --metric=none --total-kimg=10000 \
# --min-h=4 --min-w=4 --res-log2=7 --result-dir="/content/drive/My Drive/twist_moe/results/" \
# --resume-pkl='/content/network-e621.pkl'

In [ ]:
%%capture
# 00002-stylegan2-training_data-1gpu-config-f
# network-snapshot-000000
!python /content/stylegan2/run_generator.py generate-images --seeds=0-100 --truncation-psi=0.7 \
--network='/content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/network-snapshot-000006.pkl'
#%cp -av /content/stylegan2/results/00000-generate-images /content/drive/'My Drive'/twist_moe/seeds-1.0


In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import scipy
import math
import moviepy.editor
from numpy import linalg

dir = "/content/drive/My Drive/twist_moe/results/"
training_expm = "00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/"
snapshot = "network-snapshot-000006.pkl"
network_path = dir + training_expm + snapshot

def main():
    tflib.init_tf()
    _G, _D, Gs = pickle.load(open(network_path, "rb"))

    rnd = np.random
    latents_a = rnd.randn(1, Gs.input_shape[1])
    latents_b = rnd.randn(1, Gs.input_shape[1])
    latents_c = rnd.randn(1, Gs.input_shape[1])

    def circ_generator(latents_interpolate):
        radius = 40.0

        latents_axis_x = (latents_a - latents_b).flatten() / linalg.norm(latents_a - latents_b)
        latents_axis_y = (latents_a - latents_c).flatten() / linalg.norm(latents_a - latents_c)

        latents_x = math.sin(math.pi * 2.0 * latents_interpolate) * radius
        latents_y = math.cos(math.pi * 2.0 * latents_interpolate) * radius

        latents = latents_a + latents_x * latents_axis_x + latents_y * latents_axis_y
        return latents

    def mse(x, y):
        return (np.square(x - y)).mean()

    def generate_from_generator_adaptive(gen_func):
        max_step = 1.0
        current_pos = 0.0

        change_min = 10.0
        change_max = 11.0

        fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)

        current_latent = gen_func(current_pos)
        current_image = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
        array_list = []

        video_length = 1.0
        while(current_pos < video_length):
            array_list.append(current_image)

            lower = current_pos
            upper = current_pos + max_step
            current_pos = (upper + lower) / 2.0

            current_latent = gen_func(current_pos)
            current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
            current_mse = mse(array_list[-1], current_image)

            while current_mse < change_min or current_mse > change_max:
                if current_mse < change_min:
                    lower = current_pos
                    current_pos = (upper + lower) / 2.0

                if current_mse > change_max:
                    upper = current_pos
                    current_pos = (upper + lower) / 2.0


                current_latent = gen_func(current_pos)
                current_image = images = Gs.run(current_latent, None, truncation_psi=0.7, randomize_noise=False, output_transform=fmt)[0]
                current_mse = mse(array_list[-1], current_image)
            # print(current_pos, current_mse)
        return array_list

    frames = generate_from_generator_adaptive(circ_generator)
    frames = moviepy.editor.ImageSequenceClip(frames, fps=30)

    # Generate video.
    mp4_file = '/content/drive/My Drive/circular.mp4'
    mp4_codec = 'libx264'
    mp4_bitrate = '3M'
    mp4_fps = 20

    frames.write_videofile(mp4_file, fps=mp4_fps, codec=mp4_codec, bitrate=mp4_bitrate)

if __name__ == "__main__":
    main()

[MoviePy] >>>> Building video /content/drive/My Drive/circular.mp4
[MoviePy] Writing video /content/drive/My Drive/circular.mp4


100%|██████████| 1368/1368 [00:14<00:00, 93.11it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/drive/My Drive/circular.mp4 



In [ ]:
#https://stackoverflow.com/a/57378660/8773953
!pip install -U kora
from kora.drive import upload_public
url = upload_public('/content/drive/My Drive/circular.mp4')

from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")

     |████████████████████████████████| 57 kB 4.5 MB/s 
     |████████████████████████████████| 60 kB 8.8 MB/s 


In [ ]:
!zip -r /content/anime_faces_furry_pretr.zip /content/drive/My\ Drive/twist_moe

  adding: content/drive/My Drive/twist_moe/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/ (stored 0%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/submit_config.pkl (deflated 51%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/submit_config.txt (deflated 70%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/log.txt (deflated 79%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/reals.jpg (deflated 4%)
  adding: content/drive/My Drive/twist_moe/results/00000-stylegan2-high_res_anime_faces_tfrecords-1gpu-config-f/fakes_init.jpg (deflated 3%)
  adding: content/drive/My Drive/twist_moe/results/00